# Matlantis PFP 構造最適化デモ

このノートブックでは、新しく拡張した`optimization_utils.py`を使用して、
Matlantis PFPによる構造最適化を実行する方法を示します。

## 主な機能
1. **MatlantisOptimizer**: 複数のオプティマイザー（FIRE, LBFGS, BFGS）をサポート
2. **最適化履歴の追跡**: エネルギーと力の収束を自動追跡
3. **結果の可視化**: 最適化過程をグラフで確認
4. **下層原子の固定**: 表面計算などで下層を固定可能
5. **統合関数**: `optimize_structure_with_pfp`で一括実行

## 1. セットアップ

In [ ]:
import numpy as np
import sys
from pathlib import Path
from ase import Atoms
from ase.build import bulk, molecule, surface
from ase.io import read, write

# プロジェクトのutilsをインポート
sys.path.append(str(Path.cwd().parent / "LiB2_structure_ipynb"))
from utils.optimization_utils import (
    MatlantisOptimizer,
    optimize_structure_with_pfp,
    analyze_optimization_trajectory,
)

print("✓ インポート完了")

## 2. 例1: バルク結晶の最適化

歪んだAl結晶を最適化します。

In [ ]:
# 歪んだAl結晶を作成
atoms = bulk('Al', 'fcc', a=4.05).repeat((2, 2, 2))
atoms.rattle(stdev=0.1)  # ランダムに揺らす

print(f"構造: {atoms.get_chemical_formula()}")
print(f"原子数: {len(atoms)}")
print(f"セル体積: {atoms.get_volume():.2f} Å³")

In [ ]:
# 統合関数で最適化（推奨）
optimized_atoms, results = optimize_structure_with_pfp(
    atoms=atoms,
    output_dir="optimization_results",
    name="al_bulk",
    optimizer='FIRE',
    fmax=0.05,
    steps=200,
    model_version='v7.0.0',
    calc_mode='CRYSTAL_U0',
)

print("\n結果サマリー:")
print(f"  収束: {results['optimization_info']['converged']}")
print(f"  ステップ数: {results['optimization_info']['n_steps']}")
print(f"  エネルギー変化: {results['optimization_info']['energy_change']:.4f} eV")

## 3. 例2: 表面構造の最適化（下層固定）

Al(100)表面を作成し、下層を固定して最適化します。

In [ ]:
# Al(100)表面の作成
slab = surface('Al', (1, 0, 0), 4, vacuum=10.0)
slab = slab.repeat((2, 2, 1))

# 表面原子を少し揺らす
top_layer_z = slab.positions[:, 2].max()
for atom in slab:
    if atom.position[2] > top_layer_z - 2.0:
        atom.position += np.random.randn(3) * 0.05

print(f"構造: {slab.get_chemical_formula()}")
print(f"原子数: {len(slab)}")
print(f"セルサイズ: {slab.cell.diagonal()}")

In [ ]:
# 下層を固定して最適化
optimized_slab, results = optimize_structure_with_pfp(
    atoms=slab,
    output_dir="optimization_results",
    name="al_surface",
    optimizer='FIRE',
    fmax=0.05,
    steps=200,
    fix_bottom_layers=3.0,  # 最下層から3Å以内を固定
)

print("\n結果サマリー:")
print(f"  収束: {results['optimization_info']['converged']}")
print(f"  ステップ数: {results['optimization_info']['n_steps']}")
print(f"  エネルギー変化: {results['optimization_info']['energy_change']:.4f} eV")

## 4. 例3: MatlantisOptimizerクラスの直接使用

より詳細な制御が必要な場合は、`MatlantisOptimizer`クラスを直接使用できます。

In [ ]:
# テスト構造: 水分子
h2o = molecule('H2O')
h2o.center(vacuum=5.0)
h2o.set_pbc(True)

# 少し歪める
h2o.positions += np.random.randn(len(h2o), 3) * 0.05

print(f"構造: {h2o.get_chemical_formula()}")
print(f"原子数: {len(h2o)}")

In [ ]:
# オプティマイザーの初期化
optimizer = MatlantisOptimizer(
    model_version='v7.0.0',
    calc_mode='CRYSTAL_U0',
    verbose=True
)

In [ ]:
# FIRE最適化
optimized_h2o_fire, info_fire = optimizer.optimize(
    atoms=h2o,
    optimizer='FIRE',
    fmax=0.01,
    steps=100,
    trajectory_path="optimization_results/h2o_fire.traj",
)

print("\nFIRE結果:")
print(f"  収束: {info_fire['converged']}")
print(f"  ステップ数: {info_fire['n_steps']}")
print(f"  エネルギー変化: {info_fire['energy_change']:.4f} eV")

In [ ]:
# LBFGS最適化で比較
optimized_h2o_lbfgs, info_lbfgs = optimizer.optimize(
    atoms=h2o,
    optimizer='LBFGS',
    fmax=0.01,
    steps=100,
    trajectory_path="optimization_results/h2o_lbfgs.traj",
)

print("\nLBFGS結果:")
print(f"  収束: {info_lbfgs['converged']}")
print(f"  ステップ数: {info_lbfgs['n_steps']}")
print(f"  エネルギー変化: {info_lbfgs['energy_change']:.4f} eV")

## 5. 例4: 最適化結果の解析

保存されたtrajectoryファイルを解析してグラフを作成します。

In [ ]:
# Trajectoryの解析
analysis = analyze_optimization_trajectory(
    trajectory_path="optimization_results/al_bulk_optimization.traj",
    output_dir="optimization_results",
)

print("\n解析結果:")
print(f"  総ステップ数: {analysis['n_steps']}")
print(f"  初期エネルギー: {analysis['initial_energy']:.4f} eV")
print(f"  最終エネルギー: {analysis['final_energy']:.4f} eV")
print(f"  エネルギー変化: {analysis['energy_change']:.4f} eV")
print(f"  初期最大力: {analysis['initial_fmax']:.4f} eV/Å")
print(f"  最終最大力: {analysis['final_fmax']:.4f} eV/Å")

## まとめ

### 使い方の3パターン

1. **統合関数（推奨）**: `optimize_structure_with_pfp()`
   - 最もシンプル
   - 最適化、保存、解析を一括実行

2. **クラス使用**: `MatlantisOptimizer`
   - より詳細な制御が可能
   - 複数の最適化を実行する場合に効率的

3. **既存関数**: `run_matlantis_optimization()`
   - FireLBFGS統合版（Matlantis内蔵）
   - 互換性のため残されている

### 主な特徴

- ✅ 複数のオプティマイザーをサポート（FIRE, LBFGS, BFGS）
- ✅ 下層原子の自動固定機能
- ✅ 最適化履歴の自動追跡
- ✅ エネルギーと力の収束グラフ作成
- ✅ 詳細なログ出力
- ✅ エラーハンドリング

### 出力ファイル

- `*_optimization.traj`: 最適化trajectory
- `*_optimized.xyz`: 最適化後の構造
- `*_optimization_analysis.png`: 収束グラフ